In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

I have previously scraped a bunch of data about DnD monsters from [orcpub](http://www.orcpub.com/dungeons-and-dragons/5th-edition/monsters) using `MonsterScraper.py`. The first thing I want to do is load the data from the created csv file into a dataframe.

In [153]:
df = pd.read_csv('out_monsters.csv')
df.head()

,Name,Size,Type,Alignment,AC,HP,Hit Die,Speed,Str,Dex,...,Skills,Resistance,Immunity,Condition Immunity,Senses,Languages,CR,Extras,Actions,Leg Actions
0,Aboleth,Large,aberration,lawful evil,17,135,(18d10+ 36),"10 ft., swim 40 ft. ft.",21 (+5),9 (-1),...,"History +12, Perception +10",NaN,NaN,NaN,"darkvision 120 ft., passive Perception 20","Deep Speech, telepathy 120 ft.",10(5900 XP),Amphibious.Mucous Cloud.Probing Telepathy.,Multiattack.Tentacle.Tail.Enslave .,Detect.Tail Swipe.Psychic Drain .
1,Deva,Medium,celestial,lawful good,17,136,(16d8+ 64),"30 ft., fly 90 ft. ft.",18 (+4),18 (+4),...,"Insight +9, Perception +9","radiant; bludgeoning, piercing, and slashing f...",NaN,"charmed, exhaustion, frightened","darkvision 120 ft., passive Perception 19","all, telepathy 120 ft.",10(5900 XP),Angelic Weapons.Innate Spellcasting.Magic Resi...,Multiattack.Mace.Healing Touch .Change Shape.,NaN
2,Planetar,Large,celestial,lawful good,19,200,(16d10+ 112),"40 ft., fly 120 ft. ft.",24 (+7),20 (+5),...,Perception +11,"radiant; bludgeoning, piercing, and slashing f...",NaN,"charmed, exhaustion, frightened","truesight 120 ft., passive Perception 21","all, telepathy 120 ft.",16(15000 XP),Angelic Weapons.Divine Awareness.Innate Spellc...,Multiattack.Greatsword.Healing Touch .,NaN
3,Solar,Large,celestial,lawful good,21,243,(18d10+ 144),"50 ft., fly 150 ft. ft.",26 (+8),22 (+6),...,Perception +14,"radiant; bludgeoning, piercing, and slashing f...","necrotic, poison","charmed, exhaustion, frightened, poisoned","truesight 120 ft., passive Perception 24","all, telepathy 120 ft.",21(33000 XP),Angelic Weapons.Divine Awareness.Innate Spellc...,Multiattack.Greatsword.Slaying Longbow.Flying ...,Teleport.Searing Burst .Blinding Gaze .
4,Animated Armor,Medium,construct,unaligned,18,33,(6d8+ 6),25 ft. ft.,14 (+2),11 (0),...,NaN,NaN,"poison, psychic","blinded, charmed, deafened, exhaustion, fright...","blindsight 60 ft. (blind beyond this radius), ...",NaN,1(200 XP),Antimagic Susceptibility.False Appearance.,Multiattack.Slam.,NaN


Let's take a look at all of the columns available

In [154]:
print(df.columns)

Index(['Name', 'Size', 'Type', 'Alignment', 'AC', 'HP', 'Hit Die', 'Speed',
       'Str', 'Dex', 'Con', 'Int', 'Wis', 'Cha', 'Proficiency', 'Skills',
       'Resistance', 'Immunity', 'Condition Immunity', 'Senses', 'Languages',
       'CR', 'Extras', 'Actions', 'Leg Actions'],
      dtype='object')


## Basic Data Cleaning

It's easy to see that some of the data that is likely important to understanding the strength of a monster needs to be cleaned. This includes hit die, and the attributes such as strength, dexterity, etc. First, I am going to make three columns representing the number of die, the type of die, and the additional modifier.

In [155]:
# new data frame with split value columns 
#Remove the brackets from the column
df["Hit Die"] = df["Hit Die"].apply(lambda x : x.translate(str.maketrans('','','()'))) 
#split columns based on d and + delimiters
df[['N Hit Die','Dice']] = df['Hit Die'].str.split("d",expand=True)
df[['Dice', 'Offset']] = df['Dice'].str.split('+',expand=True)
df = df.drop("Hit Die",axis=1)

df.head()


,Name,Size,Type,Alignment,AC,HP,Speed,Str,Dex,Con,...,Condition Immunity,Senses,Languages,CR,Extras,Actions,Leg Actions,N Hit Die,Dice,Offset
0,Aboleth,Large,aberration,lawful evil,17,135,"10 ft., swim 40 ft. ft.",21 (+5),9 (-1),15 (+2),...,NaN,"darkvision 120 ft., passive Perception 20","Deep Speech, telepathy 120 ft.",10(5900 XP),Amphibious.Mucous Cloud.Probing Telepathy.,Multiattack.Tentacle.Tail.Enslave .,Detect.Tail Swipe.Psychic Drain .,18,10,36
1,Deva,Medium,celestial,lawful good,17,136,"30 ft., fly 90 ft. ft.",18 (+4),18 (+4),18 (+4),...,"charmed, exhaustion, frightened","darkvision 120 ft., passive Perception 19","all, telepathy 120 ft.",10(5900 XP),Angelic Weapons.Innate Spellcasting.Magic Resi...,Multiattack.Mace.Healing Touch .Change Shape.,NaN,16,8,64
2,Planetar,Large,celestial,lawful good,19,200,"40 ft., fly 120 ft. ft.",24 (+7),20 (+5),24 (+7),...,"charmed, exhaustion, frightened","truesight 120 ft., passive Perception 21","all, telepathy 120 ft.",16(15000 XP),Angelic Weapons.Divine Awareness.Innate Spellc...,Multiattack.Greatsword.Healing Touch .,NaN,16,10,112
3,Solar,Large,celestial,lawful good,21,243,"50 ft., fly 150 ft. ft.",26 (+8),22 (+6),26 (+8),...,"charmed, exhaustion, frightened, poisoned","truesight 120 ft., passive Perception 24","all, telepathy 120 ft.",21(33000 XP),Angelic Weapons.Divine Awareness.Innate Spellc...,Multiattack.Greatsword.Slaying Longbow.Flying ...,Teleport.Searing Burst .Blinding Gaze .,18,10,144
4,Animated Armor,Medium,construct,unaligned,18,33,25 ft. ft.,14 (+2),11 (0),13 (+1),...,"blinded, charmed, deafened, exhaustion, fright...","blindsight 60 ft. (blind beyond this radius), ...",NaN,1(200 XP),Antimagic Susceptibility.False Appearance.,Multiattack.Slam.,NaN,6,8,6


Now I will break apart the abilities and their modifiers while also sanitizing. It will be helpful to have a function do the work as I will need to do this multiple times.

In [156]:
def sanitize_abilities(word):
    word = word.translate(str.maketrans('','','()'))
    if '+' in word:
        return word.split('+')
    elif '-' in word:
        return word.split('-')
    else:
        return [word,0]
        
df[['Str','Str_mod']] = df['Str'].apply(lambda row: pd.Series(sanitize_abilities(row)))
df[['Dex','Dex_mod']] = df['Dex'].apply(lambda row: pd.Series(sanitize_abilities(row)))
df[['Cha','Cha_mod']] = df['Cha'].apply(lambda row: pd.Series(sanitize_abilities(row)))
df[['Con','Con_mod']] = df['Con'].apply(lambda row: pd.Series(sanitize_abilities(row)))
df[['Wis','Wis_mod']] = df['Wis'].apply(lambda row: pd.Series(sanitize_abilities(row)))
df[['Int','Int_mod']] = df['Int'].apply(lambda row: pd.Series(sanitize_abilities(row)))


The last thing I'm going to do for basic numeric values is break apart the CR and XP values. These currently look like this:

In [157]:
df.CR.head()

0     10(5900 XP)
1     10(5900 XP)
2    16(15000 XP)
3    21(33000 XP)
4       1(200 XP)
Name: CR, dtype: object

In [158]:
def clean_CR(word):
    columns = word.split('(')
    columns[1] = columns[1].split()[0]
    return columns
    
df[['CR','XP']] = df['CR'].apply(lambda row: pd.Series(clean_CR(row)))
df[['XP','CR']].head() #shows section we are interested to see if it worked

,XP,CR
0,5900,10
1,5900,10
2,15000,16
3,33000,21
4,200,1
